In [25]:
import numpy as np
import pandas as pd

In [2]:
# Load cleaned text and metadata datasets
text_df = pd.read_csv('csv/segmented_cleaned.csv')
meta_df = pd.read_excel('csv/032818_RAC_Networks_Database.xlsx')

# Extract Volume and Letter Numbers

In [20]:
def extract_vol(filename):
    '''Extract Vol for each text'''
    for name in text_df['filename']:
        if 'Volume_III' in name:
            return 3
        elif 'Vol_II'in name:
            return 2
        elif 'Vol_1' or 'Vol_I'in name:
            return 1
        elif 'Winneba_p150_152' in name:
            return 1
        elif 'Tantumkweri_Quansas_Croome' in name:
            return 3
        else:
            # Sanity check for whether anything was missed
            return None

def extract_letter_number(section_title):
    '''Extract letter number from section title'''
    return section_title.split('.')[0]

In [17]:
assert(sum(text_df['filename'].apply(extract_vol).isnull() == 0))

In [25]:
# Add columns for Vol and LetterNo 
text_df['Vol'] = text_df['filename'].apply(extract_vol)
text_df['LetterNo'] = text_df['section_title'].apply(extract_letter_number)

In [29]:
text_df.to_csv('csv/segmented_with_numbers.csv',index=False)

# Join with Metadata dataframe on Volume and Letter Number

In [4]:
# Read cleaned dataframes
text_df = pd.read_csv('csv/segmented_with_numbers.csv')
meta_df = pd.read_excel('csv/032818_RAC_Networks_Database.xlsx')

In [5]:
# Join dataframes on Letter Number and Volume
merged_df = pd.merge(meta_df, text_df, on=['Vol', 'LetterNo'])

In [9]:
merged_df.columns


Index([             u'UID',              u'Vol',         u'LetterNo',
                 u'Sender',  u'Place Sent From',        u'Ship Name',
         u'Place Going To',             u'Date',        u'Boat/Fort',
       u'RAC/Other Nation',    u'Rawlinson C. ',         u'Latitude',
              u'Longitude',            u'Note1',            u'Note2',
                  u'Note3',            u'Note4',            u'Note5',
        u'Rawlinson C. .1',      u'Unnamed: 19',      u'Unnamed: 20',
            u'Unnamed: 21',         u'filename',    u'section_title',
                   u'text',             u'date'],
      dtype='object')

In [17]:
# Filter out for a subset of the columns
merged_filtered = merged_df[[u'UID', u'Vol', u'LetterNo', u'Sender',  u'Place Sent From', u'Ship Name', u'Place Going To', u'Date',
          u'Boat/Fort', u'RAC/Other Nation', u'Rawlinson C. ', u'Rawlinson C. .1', u'text']]

In [35]:
ascii_counter = 0
total_counter = 0

def remove_non_ascii(text):
    '''Remove non-ASCII characters (mostly due to OCR parsing error)'''
    if text is not np.nan:
        return ''.join(i for i in text if ord(i)<128)
    else:
        return np.nan

# Strip away non-ASCII characters from text column
merged_filtered['text'] = merged_filtered['text'].apply(remove_non_ascii)

/Users/harryxue/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [37]:
# Export joined dataframe to csv file
merged_filtered.to_csv('csv/metadata_text_merged.csv', index=False)